## Lexical Analysis (Mandate 2 Work)

In [ ]:
# installing hugging face 'dataset' library
!pip install datasets

# importing hugging face's dataset library
from datasets import load_dataset

# load xsum dataset from hugging face
dataset = load_dataset("xsum")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
# accessing train validation and test splits of the dataset
train_dataset = dataset["train"].select(range(100))
# train_dataset = dataset["train"].select(range(100000))
# valid_dataset = dataset["validation"].select(range(1000))
# test_dataset = dataset["test"].select(range(1000))

In [ ]:
import re

def remove_links_and_emails(example):
    # Define regex patterns for links and emails
    link_pattern = r'https?://\S+|www\.\S+'
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'

    # Compile regex for efficiency if applying this to many documents
    link_regex = re.compile(link_pattern)
    email_regex = re.compile(email_pattern)

    # Remove links and emails from 'document'
    example['document'] = link_regex.sub('', example['document'])
    example['document'] = email_regex.sub('', example['document'])

    # Remove links and emails from 'summary'
    example['summary'] = link_regex.sub('', example['summary'])
    example['summary'] = email_regex.sub('', example['summary'])

    return example

In [ ]:
train_dataset_no_email_and_links = train_dataset.map(remove_links_and_emails)
# valid_dataset_no_email_and_links = valid_dataset.map(remove_links_and_emails)
# test_dataset_no_email_and_links = test_dataset.map(remove_links_and_emails)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

def sent_tokenize_example(example):
    example['document'] = sent_tokenize(example['document'])
    example['summary'] = sent_tokenize(example['summary'])
    return example

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train_sent_tokens = train_dataset_no_email_and_links.map(sent_tokenize_example)
# valid_sent_tokens = valid_dataset_no_email_and_links.map(sent_tokenize_example)
# test_sent_tokens = test_dataset_no_email_and_links.map(sent_tokenize_example)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import string
string.punctuation
def remove_punctuations(example):
    table = str.maketrans('', '', string.punctuation)
    example['document'] = [word.translate(table) for word in example['document'] if word not in string.punctuation]
    example['summary'] = [word.translate(table) for word in example['summary'] if word not in string.punctuation]

    # Remove empty strings that may result from removing punctuation
    example['document'] = [token for token in example['document'] if token]
    example['summary'] = [token for token in example['summary'] if token]
    return example

In [ ]:
train_sent_tokens = train_sent_tokens.map(remove_punctuations)
# valid_sent_tokens = valid_sent_tokens.map(remove_punctuations)
# test_sent_tokens = test_sent_tokens.map(remove_punctuations)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#word tokenization
from nltk.tokenize import word_tokenize

def word_tokenize_sent_tokens(example):
    example['document']=[word_tokenize(sentence) for sentence in example['document']]
    example['summary']=[word_tokenize(sentence) for sentence in example['summary']]
    return example

In [ ]:
train_sent_word_tokens = train_sent_tokens.map(word_tokenize_sent_tokens)
# valid_sent_word_tokens = valid_sent_tokens.map(word_tokenize_sent_tokens)
# test_sent_word_tokens = test_sent_tokens.map(word_tokenize_sent_tokens)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

# Function to convert NLTK's POS tags to the format expected by the WordNet lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def pos_tag_and_lemmatize_example(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      tagged_tokens = pos_tag(sentence)
      temp['document'].append([lemmatizer.lemmatize(wordl, tagl) for wordl, tagl in [(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]])

    for sentence in example['summary']:
      tagged_tokens = pos_tag(sentence)
      temp['summary'].append([lemmatizer.lemmatize(wordl, tagl) for wordl, tagl in [(word, get_wordnet_pos(tag)) for word, tag in tagged_tokens]])
    return temp

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(pos_tag_and_lemmatize_example)
# valid_sent_word_tokens = valid_sent_word_tokens.map(pos_tag_and_lemmatize_example)
# test_sent_word_tokens = test_sent_word_tokens.map(pos_tag_and_lemmatize_example)

Parameter 'function'=<function pos_tag_and_lemmatize_example at 0x7cd5bab4ce50> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
def lower(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      temp['document'].append([word.lower() for word in sentence])

    for sentence in example['summary']:
      temp['summary'].append([word.lower() for word in sentence])
    return temp

In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(lower)
# valid_sent_word_tokens = valid_sent_word_tokens.map(lower)
# test_sent_word_tokens = test_sent_word_tokens.map(lower)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')  # Download the list of stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(example):
    temp = {'document':[],'summary':[],'id':''}
    temp['id']=example['id']
    for sentence in example['document']:
      temp['document'].append([word for word in sentence if word not in stop_words])

    for sentence in example['summary']:
      temp['summary'].append([word for word in sentence if word not in stop_words])
    return temp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_sent_word_tokens = train_sent_word_tokens.map(remove_stopwords)
# valid_sent_word_tokens = valid_sent_word_tokens.map(remove_stopwords)
# test_sent_word_tokens = test_sent_word_tokens.map(remove_stopwords)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_word_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': [['full', 'cost', 'damage', 'newton', 'stewart', 'one', 'area', 'worst', 'affect', 'still', 'assess'], ['repair', 'work', 'ongoing', 'hawick', 'many', 'road', 'peeblesshire', 'remain', 'badly', 'affect', 'stand', 'water'], ['trains', 'west', 'coast', 'mainline', 'face', 'disruption', 'due', 'damage', 'lamington', 'viaduct'], ['many', 'business', 'householder', 'affect', 'flood', 'newton', 'stewart', 'river', 'cree', 'overflow', 'town'], ['first', 'minister', 'nicola', 'sturgeon', 'visit', 'area', 'inspect', 'damage'], ['water', 'breach', 'retain', 'wall', 'flood', 'many', 'commercial', 'property', 'victoria', 'street', 'main', 'shopping', 'thoroughfare'], ['jeanette', 'tate', 'cinnamon', 'cafe', 'badly', 'affect', 'say', 'could', 'fault', 'multiagency', 'response', 'flood', 'hit'], ['however', 'say', 'preventative', 'work', 'could', 'carry', 'ensure', 'retain', 'wall', 'fail'], ['difficult', 'think', 'much', 'publicity', 'dumfries', 'nith', 'totally', 'appreciate', 'almost

In [ ]:
def deword_tokenize(example):
  temp = {'document':[],'summary':[],'id':''}
  temp['id']=example['id']
  temp['document'] = [' '.join(sentence) for sentence in example['document']]
  temp['summary'] = [' '.join(sentence) for sentence in example['summary']]
  return temp

In [ ]:
train_sent_tokens = train_sent_word_tokens.map(deword_tokenize)
# valid_sent_tokens = valid_sent_word_tokens.map(deword_tokenize)
# test_sent_tokens = test_sent_word_tokens.map(deword_tokenize)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(train_sent_tokens):
    if i < 10:
        print(example)
    else:
        break

{'document': ['full cost damage newton stewart one area worst affect still assess', 'repair work ongoing hawick many road peeblesshire remain badly affect stand water', 'trains west coast mainline face disruption due damage lamington viaduct', 'many business householder affect flood newton stewart river cree overflow town', 'first minister nicola sturgeon visit area inspect damage', 'water breach retain wall flood many commercial property victoria street main shopping thoroughfare', 'jeanette tate cinnamon cafe badly affect say could fault multiagency response flood hit', 'however say preventative work could carry ensure retain wall fail', 'difficult think much publicity dumfries nith totally appreciate almost like neglect forget say', 'may true perhaps perspective last day', 'ready help u bit warning alarm alert go', 'meanwhile flood alert remain place across borders constant rain', 'peebles badly hit problem spark call introduce defence area', 'scottish borders council put list websi

## Syntactic Analysis (Mandate 3)

In [ ]:
!pip install transformers
import os
os.environ['HF_TOKEN'] = '<token here>'

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Load the tokenizer and model using the token for authentication
model_name = "QCRI/bert-base-multilingual-cased-pos-english"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=os.environ['HF_TOKEN'])
model = AutoModelForTokenClassification.from_pretrained(model_name, use_auth_token=os.environ['HF_TOKEN'])

# Setup the pipeline
pos_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

def transformer_pos_tag(example):
    # Initialize a structure to hold the tagged results
    tagged_sentences = []

    for sentence in example['document']:
        # Apply POS tagging
        results = pos_pipeline(sentence)
        # Store the results
        tagged_sentences.append({'sentence': sentence, 'pos_tags': results})

    # Return a dictionary with the results
    return {'document': tagged_sentences}


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_sent_tokens_10 = train_sent_tokens.select(range(10))
results = train_sent_tokens_10.map(transformer_pos_tag)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
for i, example in enumerate(results['document']):
    if i < 10:
        print(example)
    else:
        break

[{'pos_tags': [{'end': 4, 'entity': 'JJ', 'index': 1, 'score': 0.9966322779655457, 'start': 0, 'word': 'full'}, {'end': 9, 'entity': 'NN', 'index': 2, 'score': 0.9982958436012268, 'start': 5, 'word': 'cost'}, {'end': 16, 'entity': 'NN', 'index': 3, 'score': 0.9916743636131287, 'start': 10, 'word': 'damage'}, {'end': 20, 'entity': 'NN', 'index': 4, 'score': 0.9322308897972107, 'start': 17, 'word': 'new'}, {'end': 23, 'entity': 'NN', 'index': 5, 'score': 0.9946269392967224, 'start': 20, 'word': '##ton'}, {'end': 26, 'entity': 'NN', 'index': 6, 'score': 0.9595084190368652, 'start': 24, 'word': 'st'}, {'end': 28, 'entity': 'NN', 'index': 7, 'score': 0.8802160620689392, 'start': 26, 'word': '##ew'}, {'end': 31, 'entity': 'NN', 'index': 8, 'score': 0.8268393278121948, 'start': 28, 'word': '##art'}, {'end': 35, 'entity': 'CD', 'index': 9, 'score': 0.9993083477020264, 'start': 32, 'word': 'one'}, {'end': 40, 'entity': 'NN', 'index': 10, 'score': 0.9995445609092712, 'start': 36, 'word': 'area'}

In [ ]:
import spacy
from transformers import pipeline

# Load SpaCy's English model for parsing
nlp = spacy.load("en_core_web_sm")

# Setup the POS tagging pipeline with Hugging Face
pos_pipeline = pipeline("ner", model="QCRI/bert-base-multilingual-cased-pos-english")

# Example text
text = "Apple is looking at buying U.K. startup for $1 billion."

# Apply POS tagging
pos_tags = pos_pipeline(text)

# Parse the text with SpaCy
doc = nlp(text)

for token, tag_info in zip(doc, pos_tags):
    token_tag = tag_info['entity'].split('-')[-1]  # Extract the POS tag
    print(f"{token.text} -> SpaCy POS: {token.pos_}, Hugging Face POS: {token_tag}, Dependency: {token.dep_}")


Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Apple -> SpaCy POS: PROPN, Hugging Face POS: NNP, Dependency: nsubj
is -> SpaCy POS: AUX, Hugging Face POS: VBZ, Dependency: aux
looking -> SpaCy POS: VERB, Hugging Face POS: VBG, Dependency: ROOT
at -> SpaCy POS: ADP, Hugging Face POS: IN, Dependency: prep
buying -> SpaCy POS: VERB, Hugging Face POS: VBG, Dependency: pcomp
U.K. -> SpaCy POS: PROPN, Hugging Face POS: VBG, Dependency: dobj
startup -> SpaCy POS: NOUN, Hugging Face POS: NNP, Dependency: dep
for -> SpaCy POS: ADP, Hugging Face POS: ., Dependency: prep
$ -> SpaCy POS: SYM, Hugging Face POS: NNP, Dependency: quantmod
1 -> SpaCy POS: NUM, Hugging Face POS: ., Dependency: compound
billion -> SpaCy POS: NUM, Hugging Face POS: NN, Dependency: pobj
. -> SpaCy POS: PUNCT, Hugging Face POS: NN, Dependency: punct


In [ ]:
import spacy
from spacy.tokens import Doc
from transformers import AutoTokenizer

def create_custom_tokenizer(nlp, tokenizer):

    def custom_tokenizer(text):
        # Tokenize the text using the Hugging Face tokenizer
        encoded_input = tokenizer(text, return_tensors='pt', add_special_tokens=False)
        # Decode tokens into text tokens
        tokens = [tokenizer.decode(tok, skip_special_tokens=True).strip() for tok in encoded_input['input_ids'][0]]
        # Create a SpaCy Doc from the list of token texts
        return Doc(nlp.vocab, words=tokens)

    return custom_tokenizer

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Test the tokenizer
doc = nlp("Apple is looking at buying U.K. startup for $1 billion.")
print([token.text for token in doc])

['Apple', 'is', 'looking', 'at', 'buying', 'U.K.', 'startup', 'for', '$', '1', 'billion', '.']


In [ ]:
# Process the text using the SpaCy pipeline
doc = nlp(text)

from spacy import displacy

# Visualize the dependency parse
displacy.render(doc, style='dep', jupyter=True, options={'distance': 100})

In [ ]:
#entity recognition
import spacy

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Example text
text = "Apple Inc. is looking to buy a U.K. startup for $1 billion. Tim Cook announced this yesterday in San Francisco."

# Process the text
doc = nlp(text)

# Display the named entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Apple Inc. ORG
U.K. GPE
$1 billion MONEY
Tim Cook PERSON
this yesterday DATE
San Francisco GPE


In [ ]:
!pip install coreferee

In [ ]:
!python -m coreferee install en

2024-04-14 19:39:35.244325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:39:35.244393: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:39:35.245853: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee-model-en: filename=coreferee_model_en-1.0.0-py3-none-any.whl size=65422507 sha256=fc8eede031dbf318153f8d5f88ea47cfe1f241

In [ ]:
!python -m spacy download en_core_web_md


2024-04-14 19:41:50.736069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 19:41:50.736151: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 19:41:50.738141: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
import spacy
import coreferee

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('coreferee')
doc = nlp("Apple is looking at buying a startup. They are interested in its AI technology.")

print(doc._.coref_chains)


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_md' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  """Check if string maps to a package installed via pip.


[0: [0], [8], 1: [6], [12]]


In [ ]:
!pip install stanza


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 7.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manyl

In [ ]:
import stanza

# Download the English model
stanza.download('en')

# Set up the pipeline with the constituency parsing processor
nlp = stanza.Pipeline(lang='en', processors='tokenize,pos,constituency')

# Example sentence
text = "The quick brown fox jumps over the lazy dog."

# Process the text
doc = nlp(text)

# Access the constituency parse of the first sentence
constituency_parse = doc.sentences[0].constituency

# Print the parse tree
print(constituency_parse)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...
INFO:stanza:File exists: /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: constituency
INFO:stanza:Done loading processors!


(ROOT (S (NP (DT The) (JJ quick) (JJ brown) (NN fox)) (VP (VBZ jumps) (PP (IN over) (NP (DT the) (JJ lazy) (NN dog)))) (. .)))
